In [3]:
import pandas as pd

features = pd.read_csv('features.csv', index_col=[0,1])
features.head()

surname_score  given_name_score  age_score  city_score  \
    Unnamed: 1                                                           
73  541039           0.571429          0.333333       1.00         1.0   
103 1248433          0.875000          0.222222       1.00         1.0   
303 1593162          0.857143          1.000000       0.75         1.0   
    1754756          0.857143          0.111111       1.00         1.0   
358 2610721          0.666667          1.000000       1.00         1.0   

                years_since_census  is_cambridge  cardinal_prefix  is_match  
    Unnamed: 1                                                               
73  541039                       0             0                0         0  
103 1248433                      0             0                0         0  
303 1593162                      0             0                0         1  
    1754756                      0             0                0         0  
358 2610721                      0             1                0         1

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=7, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

X = features[features.columns[:-1]]
y = features[['is_match']]

In [25]:
seed = 152
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=50, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(model, X, y, cv=kfold, scoring='balanced_accuracy')
print(results.mean())

0.6963817277250113


In [49]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(class_weight='balanced',
                                max_depth=10, n_estimators=50)
results2 = cross_val_score(model2, X, y, cv=kfold, scoring='balanced_accuracy')
print(results2.mean())

C:\Users\silve\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\silve\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\silve\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\silve\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y wa

0.8672320217096336


In [50]:
model2.fit(X, y)
probas = model2.predict_proba(X)

C:\Users\silve\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.4589178356713427

In [52]:
(probas[:,1] > 0.1).astype(int).mean()

0.5370741482965932